In [1]:
# 挂载Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 安装bert4keras rouge Google权重
! pip3 install bert4keras
! pip3 install gsutil
! pip3 install numpy
! gsutil cp -r gs://t5-data/pretrained_models/mt5/small .
! gsutil cp -r gs://t5-data/vocabs/mc4.250000.100extra/sentencepiece.model .
! pip3 install rouge
! pip3 install sentencepiece

In [ ]:
# 解压数据集和权重
! unzip /content/t5_in_bert4keras-main.zip
! unzip /content/nlpcc2017.zip
# ! unzip /content/csl_title_public.zip

In [ ]:
# tensorflow2.x才能用GPU!!!
! pip3 install tensorflow==2.4.1
! pip3 install keras==2.3.1

In [8]:
from __future__ import print_function
import os
os.environ['TF_KERAS'] = '1'
import json
import random
import numpy as np
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import SpTokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
from tensorflow.keras.models import Model
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# 基本参数
max_c_len = 1024
max_t_len = 128
batch_size = 16
epochs = 40

# 模型路径
config_path = '/content/small/t5_small.json'
checkpoint_path = '/content/small/model.ckpt-1000000'
spm_path = '/content/t5_in_bert4keras-main/tokenizer/sentencepiece_cn.model'
keep_tokens_path = '/content/t5_in_bert4keras-main/tokenizer/sentencepiece_cn_keep_tokens.json'


def load_data(filename):
    D = []
    with open(filename) as f:
        for l in f:
            l = json.loads(l)
            if 'summarization' in l: 
              title = l['summarization']
            else:
              title = None
            content = l['article'].replace('<Paragraph>','\r\n')
            if len(content) > max_c_len or len(title) > max_t_len: continue
            D.append((title, content))
    return D


# 加载数据集
train_data = load_data('/content/train_with_summ.json')
valid_data = load_data('/content/evaluation_with_ground_truth.json')
random.shuffle(train_data)
train_data = train_data[:24000]
valid_data = valid_data[:100]


# 加载分词器
tokenizer = SpTokenizer(spm_path, token_start=None, token_end='</s>')
keep_tokens = json.load(open(keep_tokens_path))


class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_c_token_ids, batch_t_token_ids = [], []
        for is_end, (title, content) in self.sample(random):
            c_token_ids, _ = tokenizer.encode(content, maxlen=max_c_len)
            t_token_ids, _ = tokenizer.encode(title, maxlen=max_t_len)
            batch_c_token_ids.append(c_token_ids)
            batch_t_token_ids.append([0] + t_token_ids)
            if len(batch_c_token_ids) == self.batch_size or is_end:
                batch_c_token_ids = sequence_padding(batch_c_token_ids)
                batch_t_token_ids = sequence_padding(batch_t_token_ids)
                yield [batch_c_token_ids, batch_t_token_ids], None
                batch_c_token_ids, batch_t_token_ids = [], []


class CrossEntropy(Loss):
    """交叉熵作为loss，并mask掉输入部分
    """
    def compute_loss(self, inputs, mask=None):
        y_true, y_pred = inputs
        y_true = y_true[:, 1:]  # 目标token_ids
        y_mask = K.cast(mask[1], K.floatx())[:, :-1]  # 解码器自带mask
        y_pred = y_pred[:, :-1]  # 预测序列，错开一位
        loss = K.sparse_categorical_crossentropy(y_true, y_pred)
        loss = K.sum(loss * y_mask) / K.sum(y_mask)
        return loss


t5 = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    keep_tokens=keep_tokens,
    model='t5.1.1',
    return_keras_model=False,
    name='T5',
)

encoder = t5.encoder
decoder = t5.decoder
model = t5.model
model.summary()

output = CrossEntropy(1)([model.inputs[1], model.outputs[0]])

model = Model(model.inputs, output)
model.compile(optimizer=Adam(1e-4))


class AutoTitle(AutoRegressiveDecoder):
    """seq2seq解码器
    """
    @AutoRegressiveDecoder.wraps(default_rtype='probas')
    def predict(self, inputs, output_ids, states):
        c_encoded = inputs[0]
        return decoder.predict([c_encoded, output_ids])[:, -1]

    def generate(self, text, topk=1):
        c_token_ids, _ = tokenizer.encode(text, maxlen=max_c_len)
        c_encoded = encoder.predict(np.array([c_token_ids]))[0]
        output_ids = self.beam_search([c_encoded], topk)  # 基于beam search
        return tokenizer.decode([int(i) for i in output_ids])


# 注：T5有一个很让人不解的设置，它的<bos>标记id是0，即<bos>和<pad>其实都是0
autotitle = AutoTitle(start_id=0, end_id=tokenizer._token_end_id, maxlen=128)

def just_show():
    s1 = u'资料图：空军苏27/歼11编队，日方称约40架中国军机在23日8艘海监船驱赶日本船队期间出现在钓鱼岛附近空域日本《产经新闻》4月27日报道声称，中国8艘海监船相继进入钓鱼岛12海里执法的4月23日当天，曾有40多架中国军机出现在钓鱼岛海域周边空域，且中方军机中多半为战斗机，包括中国空军新型战机苏-27和苏-30。日本《产经新闻》声称中国军机是想通过不断的逼近，让日本航空自卫队的战机飞行员形成疲劳。日本政府高官还称：“这是前所未有的威胁。”针对日本媒体的报道，国防部官员在接受环球网采访时称，中国军队飞机在本国管辖海域上空进行正常战备巡逻，日方却颠倒黑白、倒打一耙，肆意渲染“中国威胁”。国防部官员应询表示：4月23日，日方出动多批次F-15战斗机、P3C反潜巡逻机等，对中方正常战备巡逻的飞机进行跟踪、监视和干扰，影响中方飞机正常巡逻和飞行安全。中方对此坚决采取了应对措施。中国军队飞机在本国管辖海域上空进行正常战备巡逻，日方却颠倒黑白、倒打一耙，肆意渲染“中国威胁”。国防部官员称，需要指出的是，今年年初以来，日方不断挑衅，制造事端，并采取“恶人先告状”的手法，抹黑中国军队。事实证明，日方才是地区和平稳定的麻烦制造者。我们要求日方切实采取措施，停止故意制造地区紧张局势的做法。'
    s2 = u'中新网5月26日电/r/n据外媒报道，世界卫生组织日前发布了一份报告，指出自杀已经取代难产，成为全球年轻女性的头号杀手。报道称，根据报告提供的数据显示，东南亚平均每10万名年龄介于15至19岁的女性死者中，就有27.92人死于自杀，男性则为21.41人；欧洲和美洲分别为6.15人及4.72人，全球平均值则是11.73人。报道指出，多年来，难产死亡一直是这个年龄层女性丧命的最主要原因，然而在过去10年，自杀取代难产死亡，成为全球年轻女性死亡的最主要原因。报告将全球分为美洲、东南亚、中东、欧洲、非洲及西太平洋6大地区，自杀唯独在非洲未有列入5大杀手之内，原因是当地难产和艾滋病死因占绝大多数。在东南亚，自杀占少女死因的比率也较其他死因高两倍。专家分析指出，造成这种结果的原因是当地性别歧视较严重。'
    for s in [s1, s2]:
        print(u'生成摘要:', autotitle.generate(s))
    print()

class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.rouge = Rouge()
        self.smooth = SmoothingFunction().method1
        self.best_bleu = 0.

    def on_epoch_end(self, epoch, logs=None):
        metrics = self.evaluate(valid_data)  # 评测模型
        if metrics['bleu'] > self.best_bleu:
            self.best_bleu = metrics['bleu']
            model.save_weights('./gdrive/MyDrive/T5_Bert4Keras/best_model.weights')  # 保存模型
        metrics['best_bleu'] = self.best_bleu
        print('valid_data:', metrics)
        just_show()

    def evaluate(self, data, topk=1):
        total = 0
        rouge_1, rouge_2, rouge_l, bleu = 0, 0, 0, 0
        for title, content in tqdm(data):
            total += 1
            title = ' '.join(title).lower()
            pred_title = ' '.join(autotitle.generate(content, topk)).lower()
            if pred_title.strip():
                scores = self.rouge.get_scores(hyps=pred_title, refs=title)
                rouge_1 += scores[0]['rouge-1']['f']
                rouge_2 += scores[0]['rouge-2']['f']
                rouge_l += scores[0]['rouge-l']['f']
                bleu += sentence_bleu(
                    references=[title.split(' ')],
                    hypothesis=pred_title.split(' '),
                    smoothing_function=self.smooth
                )
        rouge_1 /= total
        rouge_2 /= total
        rouge_l /= total
        bleu /= total
        return {
            'rouge-1': rouge_1,
            'rouge-2': rouge_2,
            'rouge-l': rouge_l,
            'bleu': bleu,
        }


if __name__ == '__main__':
    model.load_weights('./gdrive/MyDrive/T5_Bert4Keras/best_model.weights')
    evaluator = Evaluator()
    train_generator = data_generator(train_data, batch_size)

    model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        callbacks=[evaluator]
    )

else:
    model.load_weights('./gdrive/MyDrive/T5_Bert4Keras/best_model.weights')

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder-Input-Token (InputLayer [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 512)    16690176    Encoder-Input-Token[0][0]        
__________________________________________________________________________________________________
Encoder-Embedding-Dropout (Drop (None, None, 512)    0           Embedding-Token[0][0]            
__________________________________________________________________________________________________
Encoder-Transformer-0-MultiHead (None, None, 512)    512         Encoder-Embedding-Dropout[0][0]  
____________________________________________________________________________________________

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [04:55<00:00,  2.96s/it]


valid_data: {'rouge-1': 0.68541080629126, 'rouge-2': 0.5695124734734415, 'rouge-l': 0.6824195134568879, 'bleu': 0.5011350229574261, 'best_bleu': 0.5011350229574261}
生成摘要: 日媒称40多架中国军机在钓鱼岛附近空域出现,且多半为战斗机,日方称是前所未有的威胁。
生成摘要: 世界卫生组织发布报告称,在过去10年,自杀取代难产死亡,成为全球年轻女性死亡的最主要原因

Epoch 2/40
1500/1500 [==============================] - 690s 460ms/step - loss: 0.2930


100%|██████████| 100/100 [04:25<00:00,  2.65s/it]


...

100%|██████████| 100/100 [04:31<00:00,  2.72s/it]


valid_data: {'rouge-1': 0.6745976128812384, 'rouge-2': 0.5641596309869941, 'rouge-l': 0.6739507323499487, 'bleu': 0.49653784579138377, 'best_bleu': 0.5017966186136636}
生成摘要: 日媒称40架中国军机昨日驱逐日船,在8艘海监船执法,日军机多半为战斗机,日政府高官称这是前所未有的威胁。
生成摘要: 世界卫生组织发布报告称,在过去10年,自杀取代难产死亡,成为全球年轻女性死亡的最主要原因

Epoch 18/40
1500/1500 [==============================] - 681s 454ms/step - loss: 0.1494


100%|██████████| 100/100 [04:30<00:00,  2.70s/it]


valid_data: {'rouge-1': 0.676809436309115, 'rouge-2': 0.563547592604703, 'rouge-l': 0.6742741783340045, 'bleu': 0.4995787408418925, 'best_bleu': 0.5017966186136636}
生成摘要: 日媒称40余架中国军机在钓鱼岛海域出现钓鱼岛附近空域,日方称曾多次驱逐日本船队;日方称系日媒多次挑衅。
生成摘要: 世界卫生组织发布报告称,在过去10年,自杀取代难产死亡,成为全球年轻女性死亡的最主要原因

Epoch 19/40
 291/1500 [====>.........................] - ETA: 9:06 - loss: 0.1415

KeyboardInterrupt: ignored